In [2]:
import os
from pathlib import Path

import torch
from torch import nn
from torch.utils.data import DataLoader, Subset, random_split
from torchvision import datasets

from mllib.pytorch.files import save_results_as_csv
from mllib.pytorch.plot import save_plot_results
from mllib.pytorch.train import train
from mllib.utils.dvc import load_params
from mllib.utils.io import make_folder
from mllib.utils.logs import get_logger
from mllib.utils.pkl import save_data_as_pkl
from src.train.models import get_simple_image_transform, get_torch_model
from src.train.dataset import MelDataset
from src.utils.files import get_label_dict


params = load_params()
logger = get_logger("TRAIN", log_level=params["base"]["log_level"])

zip_path = Path(params["data"]["train_zip"])
train_path = Path(params["data"]["processed_train_dir"])
model_path = Path(params["train"]["model"])
n_images = params["train"]["number_images"]

resluts_path_csv = Path(params["results"]["train_results_csv"])
resluts_path_png = Path(params["results"]["train_results_png"])
device = "cuda" if torch.cuda.is_available() else "cpu"

image_transform = get_simple_image_transform()

label_dict = get_label_dict(zip_path=zip_path, prefix="", postfix=".png")
train_dataset = MelDataset(
    target_dir=train_path, labels_dict=label_dict, transform=image_transform
)
# train_dataset = datasets.ImageFolder(
#     root=train_path, transform=image_transform, target_transform=None
# )

c:\Users\anton\.virtualenvs\06_bio-z4wlwYlg\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
from src.train.models import get_simple_image_transform

label_dict = get_label_dict(zip_path=zip_path, prefix="", postfix=".png")
train_dataset = MelDataset(
    target_dir=train_path, labels_dict=label_dict, transform=image_transform
)

from torchvision import transforms

img = train_dataset.load_image(0)
tr = get_simple_image_transform()

tr(img).shape,
img = transforms.Resize((64, 64))(img)
transforms.ToTensor()(img).size()

torch.Size([4, 64, 64])

In [28]:
from PIL import Image

img = Image.open("data/processed/test_processed/0a2df9e11b2064f934fc30036888166f.png")
img
img = transforms.Resize((64, 64))(img)
transforms.ToTensor()(img).size()
img.mode

'RGBA'

In [ ]:
if n_images:
    train_dataset = Subset(train_dataset, torch.arange(0, n_images))
    logger.info("Number images to process {}".format(n_images))

# Split data

train_data, test_data = random_split(
    train_dataset,
    lengths=[0.8, 0.2],
    generator=torch.Generator().manual_seed(params["base"]["seed"]),
)

logger.info(
    "Train data size {}, test data size {}".format(len(train_data), len(test_data))
)

# Create DataLoaders
train_dataloader = DataLoader(
    dataset=train_data, batch_size=24, shuffle=True, num_workers=0
)
test_dataloader = DataLoader(
    dataset=test_data, batch_size=24, shuffle=12, num_workers=0
)

In [ ]:
%load_ext autoreload
%autoreload 2

# Load Libs
from src.stages.train import train_model
from mllib.pytorch.plot import plot_results


results = train_model()
plot_results(results=results)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2024-05-01 15:57:23,791 — TRAIN — INFO — Number images to process 100
2024-05-01 15:57:23,791 — TRAIN — INFO — Train data size 80, test data size 20
2024-05-01 15:57:24,264 — TRAIN — INFO — TRAIN STARTS ...


  0%|          | 0/15 [01:03<?, ?it/s]


RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[24, 4, 64, 64] to have 3 channels, but got 4 channels instead